### Instalando librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix
import keras

from keras.models import Sequential
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
tf.__version__

'2.2.0'

### Lectura de datos csv

In [3]:
train_df=pd.read_csv('../data/mitbih_train.csv',header=None)
test_df=pd.read_csv('../data/mitbih_test.csv',header=None)

### Preprocesamiento

In [4]:
from sklearn.utils import resample

df_1=train_df[train_df[187]==1]
df_2=train_df[train_df[187]==2]
df_3=train_df[train_df[187]==3]
df_4=train_df[train_df[187]==4]
df_0=(train_df[train_df[187]==0]).sample(n=20000,random_state=42)

df_1_upsample=resample(df_1,replace=True,n_samples=20000,random_state=123)
df_2_upsample=resample(df_2,replace=True,n_samples=20000,random_state=124)
df_3_upsample=resample(df_3,replace=True,n_samples=20000,random_state=125)
df_4_upsample=resample(df_4,replace=True,n_samples=20000,random_state=126)

train_df=pd.concat([df_0,df_1_upsample,df_2_upsample,df_3_upsample,df_4_upsample])

### Shaping

In [5]:
target_train = train_df[187]
target_test = test_df[187]

y_train = to_categorical(target_train)
y_test = to_categorical(target_test)

In [6]:
y_test.shape

(21892, 5)

In [7]:
X_train=train_df.iloc[:,:186].values
X_test=test_df.iloc[:,:186].values

print('Antes del reshaping: ', X_train.shape)

X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)

print('Despues del reshaping: ', X_train.shape)

Antes del reshaping:  (100000, 186)
Despues del reshaping:  (100000, 186, 1)


### Modelo

In [8]:
from keras.layers import Bidirectional, LSTM

In [ ]:
model = Sequential()

model.add(Bidirectional(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]))))

model.add(Dropout(rate = 0.25))

model.add(Dense(32))
model.add(Dense(5, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=8),
                 ModelCheckpoint(filepath='lstm_model.h5', monitor='val_loss', save_best_only=True)]

history=model.fit(X_train, y_train,epochs=40,callbacks=callbacks, batch_size=1024,validation_data=(X_test,y_test))

Epoch 1/40
98/98 [==============================] - 219s 2s/step - loss: 0.5016 - accuracy: 0.3004 - val_loss: 0.4931 - val_accuracy: 0.1428
Epoch 2/40
98/98 [==============================] - 214s 2s/step - loss: 0.3445 - accuracy: 0.6308 - val_loss: 0.4007 - val_accuracy: 0.4748
Epoch 3/40
98/98 [==============================] - 213s 2s/step - loss: 0.2767 - accuracy: 0.7267 - val_loss: 0.3466 - val_accuracy: 0.6181
Epoch 4/40
98/98 [==============================] - 216s 2s/step - loss: 0.2315 - accuracy: 0.7659 - val_loss: 0.2773 - val_accuracy: 0.7533
Epoch 5/40
88/98 [=========================>....] - ETA: 20s - loss: 0.2059 - accuracy: 0.7942